In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplleaflet

from geopy.distance import geodesic
import os

from tqdm.auto import tqdm  # for notebooks

%matplotlib inline

In [33]:
data_dir = '../data/'
mamografo_dataset_name = 'RF- Mamógrafos.csv'
use_cols = ['COD_MUN','2018/Dez'] # obtendo os dados mais recentes
municipios_dataset_name = 'dados_mun.csv'
tqdm.pandas()

In [134]:
def dist_to_near_mamografo(row):
    '''
    Retorna a menor distancia a um mamografo, dada a latitude e longitude de uma cidade
    '''
    distancias = cities_mamografo.apply(lambda x: geodesic((x['LATITUDE'], x['LONGITUDE']), (row.LATITUDE, row.LONGITUDE)).km , axis=1)
    index_min = distancias.idxmin()
    near_city = cities_mamografo.iloc[index_min]
    distancia = geodesic((near_city.LATITUDE, near_city.LONGITUDE), (row.LATITUDE, row.LONGITUDE)).km
    return near_city.COD_MUN, distancia
def population_around_mamografo(codigo_mun):
    '''
    Retorna a soma das populacoes das cidades ao redor de um mamografo
    '''
    cidades_proximas = municipios[municipios.COD_PROX == codigo_mun]
    return cidades_proximas.POPULACAO.astype(int).sum()

In [35]:
# Lendo os dataframes:

mamografo = pd.read_csv(os.path.join(data_dir, mamografo_dataset_name), encoding='latin', index_col=False, 
                     error_bad_lines=False, low_memory=False, usecols=use_cols)

municipios = pd.read_csv(os.path.join(data_dir, municipios_dataset_name), encoding='latin', index_col=0, 
                     error_bad_lines=False, low_memory=False)

municipios['COD_MUN'] = municipios['COD_MUN'].apply(lambda x: (int) ((x - x%10)/10))

In [36]:
cities_mamografo = pd.merge(mamografo[mamografo['2018/Dez'] > 0], municipios, on='COD_MUN', how='left')

In [45]:
municipios = municipios.assign(DISTANCIAS = municipios.progress_apply(lambda row: dist_to_near_mamografo(row)[1], axis=1), COD_PROX = municipios.progress_apply(lambda row: dist_to_near_mamografo(row)[0], axis=1))

In [196]:
UF_analisado = 'RS'
distancia_max = 70
populacao_max = 100000
cidades_no_estado = municipios[municipios.UF == UF_analisado]
cidades_no_estado  = cidades_no_estado[cidades_no_estado.DISTANCIAS != 0]
proximos, distantes = cidades_no_estado[cidades_no_estado.DISTANCIAS < distancia_max], cidades_no_estado[cidades_no_estado.DISTANCIAS > distancia_max]
mamografo_no_estado = cities_mamografo[cities_mamografo.UF == UF_analisado]
mamografo_no_estado = mamografo_no_estado.assign(POP_ATENDIDA = mamografo_no_estado.COD_MUN.progress_apply(lambda cod: population_around_mamografo(cod)))
mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA < populacao_max].count()
pop_ok, pop_n_ok = mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA < populacao_max], mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA > populacao_max]

In [197]:
plt.scatter(distantes.LONGITUDE, distantes.LATITUDE, color='r' , s = 70) # plotar cidades com mamografos distantes
plt.scatter(proximos.LONGITUDE, proximos.LATITUDE, color='b',s = 70) # plotar cidades com mamografos proximos
plt.scatter(pop_ok.LONGITUDE, pop_ok.LATITUDE, marker='^', color='yellowgreen', s = 400) # plotar cidades em que o numero de mamografos sao adequados
plt.scatter(pop_n_ok.LONGITUDE, pop_n_ok.LATITUDE, marker='^', color='darkolivegreen', s = 400) #plotar cidades em que o numero de mamografos nao sao adequados
mplleaflet.show()

In [ ]:
# analise de populacao por mamografo em cada estado
